In [ ]:
import os

import django
from django.core.exceptions import ObjectDoesNotExist
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt
from random import randint

# for easier visualization it is recommended to use pandas to render data...
# if pandas is not installed, you may install it with this command: pip install -U pandas
# pandas is not a dependency of django_ledger...

# Set your django settings module if needed...
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_ledger_starter.settings'

# if using jupyter notebook need to set DJANGO_ALLOW_ASYNC_UNSAFE as "true"
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

# change your working directory as needed...
os.chdir('../')

django.setup()

from django_ledger.models import EntityModel, ChartOfAccountModel, AccountModel, DEBIT, CREDIT
from django_ledger.io import roles
from django_ledger import __version__ as DJL_VERSION
from django.contrib.auth import get_user_model

print(f'Using Django Ledger Version v{DJL_VERSION}')

# Account Model
- An Account Model is a bucket of money that has a specific role in the financial ecosystem of an entity.
- Its balance represents amount of money present in the financial records of the Entity for a given accounting period.
- The total balance of an Account is increased or decreased by adding DEBIT or CREDIT Transactions to its ledger.
- Depending on the Balance Type of the Account, a DEBIT or a CREDIT will have different impacts on its total balance.
- Examples of Accounts Models are *Bank Accounts, Credit Cards, Mortgage Balances, Accounts Payable, etc*.

 <img src="./assets/djl_core_model_account.png" alt="Django Ledger Core Model" width="1200" height="600"> 

#### For comprehensive documentation on the Account Model API you can go to the Official Django Ledger Documentation Site.
[Account Model Documentation](https://django-ledger.readthedocs.io/en/latest/models.html#module-django_ledger.models.accounts)

# Get Your Entity Administrator UserModel

In [ ]:
# change this to your preferred django username...
MY_USERNAME = 'ceo_user'
MY_PASSWORD = 'NeverUseMe|VeryInsecure!'
UserModel = get_user_model()

try:
    ceo_user = UserModel.objects.get(username__exact=MY_USERNAME)
except:
    ceo_user = UserModel(username=MY_USERNAME)
    ceo_user.set_password(MY_PASSWORD)
    ceo_user.save()

# Get or Create an Entity Model

In [ ]:
ENTITY_NAME = f'Account Model Notebook {randint(1000, 9999)}, LLC'

try:
    entity_model: EntityModel = EntityModel.objects.for_user(
        user_model=ceo_user).get(name__exact=ENTITY_NAME)
except ObjectDoesNotExist:
    entity_model = EntityModel.create_entity(
        name=ENTITY_NAME,
        admin=ceo_user,
        use_accrual_method=False,
        fy_start_month=1
    )

entity_model

# Let's Create a Chart of Accounts to Work With

In [ ]:
coa_model: ChartOfAccountModel = entity_model.create_chart_of_accounts(
    coa_name='Account Model Notebook CoA',
    commit=True,
    assign_as_default=True
)

In [ ]:
coa_model

# The Root Accounts Are Automatically Created
- Not used for transactions.

In [ ]:
coa_tree = coa_model.get_coa_account_tree()
G = nx.from_dict_of_dicts({
    f"{l1['data']['code']}\n{l1['data']['name']}\nLocked: {l1['data']['locked']}": {
        f"{l2['data']['code']}\n{l2['data']['name']}\nLocked: {l2['data']['locked']}": {
            'weight': 1
        } for l2 in l1['children']
    } for l1 in coa_tree
}, create_using=nx.DiGraph)

options = {
    'font_size': 14,
    # 'font_color': 'red',
    'node_size': 8000,
    'node_color': 'white',
    'edgecolors': 'black',
    'linewidths': 2,
    'width': 3,
}

fig, ax = plt.subplots(figsize=(12, 12))
ax.margins(0.2, 0.2)
nx.draw_networkx(G, pos=nx.planar_layout(G), **options)
ax.set_title(f'Django Ledger | Initial Code of Accounts Structure')

# Create Your First Account

### Beware of the Balance Type

- Define the new account balance type: DEBIT/CREDIT
- Define the new account code
- Define the Account Role

- BT: DEBIT -> DEBIT Increase Blance
- BT: DEBIT -> CREDIT Decrease Blance

- BT: CREDIT -> DEBIT Decrease Blance
- BT: CREDIT -> CREDIT Increase Blance

In [ ]:
# roles.INCOME_OPERATIONAL

In [ ]:
# pd.DataFrame(roles.VALID_ROLES)

### Select the Appropriate Account Role

- CASH/BANK ACCOUNT -> roles.ASSET_CA_CASH
- CREDIT CARD -> roles.LIABILITY_CL_ACC_PAYABLE

- EXPENSE -> roles.EXPENSE_OPERATIONAL
- INCOME -> roles.INCOME_OPERATIONAL

## Direct Model Instantiation Will Not Work

In [ ]:
account_model_cash = AccountModel(
    code='1101',
    name='Big Bank Checking Account',
    role=roles.ASSET_CA_CASH,
    balance_type=DEBIT,
    coa_model=coa_model,
    active=False
)
account_model_cash.clean()

# This is because accounts need to be inserted into the account tree model.
account_model_cash.save()

## Use the provided Entity Model API

In [ ]:
account_model_cash = entity_model.create_account(
    code='1101',
    name='Big Bank Checking Account',
    role=roles.ASSET_CA_CASH,
    balance_type=DEBIT,
    coa_model=coa_model,
    active=False
)

In [ ]:
account_model_cash

## Let's create an account of type Expense

In [ ]:
account_model_expense = entity_model.create_account(
    code='1102',
    name='Office Expenses',
    role=roles.EXPENSE_OPERATIONAL,
    balance_type=DEBIT,
    coa_model=coa_model,
    active=False
)

## Account Codes MUST Follow The Convention:
- **1**: Assets
- **2**: Liabilities
- **3**: Capital/Equity
- **4**: Income
- **5**: COGS
- **6**: Expense

In [ ]:
account_model_expense = entity_model.create_account(
    code='6102',
    name='Office Expenses',
    role=roles.EXPENSE_OPERATIONAL,
    balance_type=DEBIT,
    coa_model=coa_model,
    active=False
)

In [ ]:
account_model_expense

### Account numbers are unique for each Chart of Accounts

In [ ]:
entity_model.create_account(
    code='6102',
    name='Office Expenses #2',
    role=roles.EXPENSE_OPERATIONAL,
    balance_type=DEBIT,
    coa_model=coa_model,
    active=False
)

# Get Your Account Models

### Option 1: Use the CoA

In [ ]:
account_model_qs = coa_model.get_non_root_coa_accounts_qs()
pd.DataFrame(account_model_qs.values(
    'coa_model__name',
    'coa_model__slug',
    'code',
    'name',
    'balance_type',
    'role',
    'locked',
    'active'
))

### Option 2: Use the Entity Instance
- Uses default CoA if coa_model is None

In [ ]:
account_model_qs = entity_model.get_coa_accounts()
pd.DataFrame(account_model_qs.values())

In [ ]:
account_model_qs = entity_model.get_coa_accounts(
    coa_model=coa_model,
    active=False)
pd.DataFrame(account_model_qs.values())

In [ ]:
account_model_qs = entity_model.get_default_coa_accounts(active=False)
pd.DataFrame(account_model_qs.values())

### Gets ALL the accounts, regardless of the CoA Model!

In [ ]:
account_model_qs = entity_model.get_all_accounts(active=False)
pd.DataFrame(
    account_model_qs.values(
        'coa_model__name',
        'coa_model__slug',
        'code',
        'name',
        'balance_type',
        'role',
        'locked',
        'active'
    ))

In [ ]:
# coa_model.

# Account Status

#### Active Status

In [ ]:
account_model_cash.is_active()

In [ ]:
account_model_cash.can_activate()

In [ ]:
account_model_cash.activate()
account_model_expense.activate()

In [ ]:
account_model_qs = entity_model.get_coa_accounts(coa_model=coa_model)
pd.DataFrame(account_model_qs.values())

# Account Methods

In [ ]:
pd.Series({
    'is_asset': account_model_cash.is_asset(),
    'is_liability': account_model_cash.is_liability(),
    'is_capital': account_model_cash.is_capital(),
    'is_income': account_model_cash.is_income(),
    'is_cogs': account_model_cash.is_cogs(),
    'is_expense': account_model_cash.is_expense(),
    'can_activate': account_model_cash.can_activate(),
    'can_deactivate': account_model_cash.can_deactivate(),
    'is_locked': account_model_cash.is_locked(),
}, name=account_model_cash)

In [ ]:
pd.Series({
    'is_asset': account_model_expense.is_asset(),
    'is_liability': account_model_expense.is_liability(),
    'is_capital': account_model_expense.is_capital(),
    'is_income': account_model_expense.is_income(),
    'is_cogs': account_model_expense.is_cogs(),
    'is_expense': account_model_expense.is_expense(),
    'can_activate': account_model_expense.can_activate(),
    'can_deactivate': account_model_expense.can_deactivate(),
    'is_locked': account_model_expense.is_locked(),
}, name=account_model_expense)

# Account Model UI

In [ ]:
URL = coa_model.get_account_list_url()
URL = f'http://localhost:8000{URL}'
URL

# Automatically Populate a CoA With the Default Chart of Accounts from Django Ledger

In [ ]:
coa_model_2 = entity_model.create_chart_of_accounts(
    coa_name='Tutorial CoA'
)

In [ ]:
coa_model_2.get_non_root_coa_accounts_qs()

In [ ]:
entity_model.set_default_coa(coa_model_2)

In [ ]:
entity_model.populate_default_coa()

In [ ]:
coa_model_2.get_non_root_coa_accounts_qs()